# Chapter 5
# Applications
_Version: February 14, 2022, see_ [PyEcon.org](https://pyecon.org).

In this chapter, we see some more advanced applications of the covered topics and their combinations:
- Further time series,
- Financial applications,
- Nonlinear optimization methods.

## Section 5.1
## Time series

### Datetime creation

In [ ]:
from datetime import datetime
now = datetime.now()
now
now.day
now.hour

### Datetime representation

In [ ]:
holiday = datetime(2020, 12, 24, 8, 30)
holiday
exam = datetime(2020, 12, 9, 10)
print("The exam will be on the " + "{:%Y-%m-%d}".format(exam))

### Datetime difference

In [ ]:
from datetime import timedelta
delta = exam - now
delta
print("The exam will take place in " + str(delta.days) + " days.")
now
now + timedelta(10, 120)

### Convert Datetime

In [ ]:
stamp = datetime(2020, 4, 12)
stamp
print("German date format: " + stamp.strftime("%d.%m.%Y"))
val = "2020-5-5"
d = datetime.strptime(val, "%Y-%m-%d")
d

### Converting examples

In [ ]:
val = "31.01.2012"
d = datetime.strptime(val, "%d.%m.%Y")
d
now.strftime("Today is %A and we are in week %W of the year %Y.")
now.strftime("%c")

### Date ranges

In [ ]:
import pandas as pd
index = pd.date_range("2020-01-01", now)
index[0:2]
index[15:16]
index = pd.date_range("2020-01-01", now, freq="M")
index[0:2]

### Resample date ranges

In [ ]:
import numpy as np
start = datetime(2016, 1, 1)
ind = pd.date_range(start, now)
numbers = np.arange((now - start).days + 1)
df = pd.DataFrame(numbers, index=ind)

## Section 5.2
## Moving window

### Rolling mean

In [ ]:
import matplotlib.pyplot as plt
amazon = pd.read_csv("data/amzn.csv", index_col=0,
                     parse_dates=True)["Adj Close"]
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
ax.set_ylabel("price")
amazon.plot(ax=ax, label="Amazon")
amazon.rolling(window=20).mean().plot(ax=ax, label="Rolling mean")
ax.legend(loc="best")
ax.set_title("Amazon price and rolling mean", fontsize=25)
fig.savefig("out/amzn.pdf")

### Standard deviation

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
pfizer = pd.read_csv("data/pfe.csv", index_col=0,
                     parse_dates=True)["Adj Close"]
pg = pd.read_csv("data/pg.csv", index_col=0,
                 parse_dates=True)["Adj Close"]
prices = pd.DataFrame(index=amazon.index)
prices["amazon"] = pd.DataFrame(amazon)
prices["pfizer"] = pd.DataFrame(pfizer)
prices["pg"] = pd.DataFrame(pg)
prices_std = prices.rolling(window=20).std()
prices_std.plot(ax=ax)
ax.set_title("Standard deviation", fontsize=25)
fig.savefig("out/std.pdf")

### Logarithmic standard deviation

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
prices_std.plot(ax=ax, logy=True)
ax.set_title("Logarithmic standard deviation", fontsize=25)
fig.savefig("out/std_log.pdf")

### Exponentially weighted functions

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
amazon.rolling(window=40).mean().plot(ax=ax, label="Rolling mean")
amazon.ewm(span=40).mean().plot(ax=ax, label="Exp mean",
                                linestyle="--", color="red")
amazon.plot(ax=ax, label="Amazon price")
ax.legend(loc="best")
ax.set_title("Exponentially weighted functions", fontsize=25)
fig.savefig("out/mean.pdf")

### Percentage change

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
returns = prices.pct_change()
returns.head()
returns.plot(ax=ax)
ax.set_title("Returns", fontsize=25)
fig.savefig("out/returns.pdf")

### Correlation

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
DJI = pd.read_csv("data/dji.csv", index_col=0,
                  parse_dates=True)["Adj Close"]
DJI_ret = DJI.pct_change()
corr = returns.rolling(window=20).corr(DJI_ret)
corr.plot(ax=ax)
ax.grid()
ax.set_title("20 days correlation", fontsize=25)
fig.savefig("out/corr.pdf")

## Section 5.3
## Financial applications

### Returns

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
ret_index = (1 + returns).cumprod()
stocks = ["amazon", "pfizer", "pg"]
for i in stocks:
    ret_index[i][0] = 1
ret_index.tail()
ret_index.plot(ax=ax)
ax.set_title("Cumulative returns", fontsize=25)
fig.savefig("out/cumret.pdf")

### Monthly returns

In [ ]:
returns_m = ret_index.resample("BM").last().pct_change()
returns_m.head()

### Volatility

In [ ]:
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
vola = returns.rolling(window=20).std() * np.sqrt(20)
vola.plot(ax=ax)
ax.set_title("Volatility", fontsize=25)
fig.savefig("out/vola.pdf")

### Describe

In [ ]:
prices.describe()

### Histogram

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 8), sharex=True)
for i in range(3):
    ax[i].set_title(stocks[i])
    returns[stocks[i]].hist(ax=ax[i], bins=50)
fig.savefig("out/return_hist.pdf")

### Regression data

In [ ]:
import statsmodels.api as sm

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
Y = np.array(amazon.loc["2018-1-1":"2018-1-15"].tolist())
X = np.arange(len(Y))
ax.scatter(x=X, y=Y, marker="o", color="red")
fig.savefig("out/reg_data.pdf")

### Regression

In [ ]:
X_reg = sm.add_constant(X)
res = sm.OLS(Y, X_reg).fit()
b, a = res.params
ax.plot(X, a * X + b)
fig.savefig("out/ols.pdf")

## Section 5.4
## Optimization

### Newton-Raphson requirements

In [ ]:
def f(x):
    return 3 * x**3 + 3 * x**2 - 5 * x


def df(x):
    return 9 * x**2 + 6 * x - 5

### Newton-Raphson

In [ ]:
def newton_raphson(fun, dfun, x0, e):
    delta = abs(fun(x0))
    while delta > e:
        ax.scatter(x0, f(x0), color="red", s=80)
        x0 = x0 - fun(x0) / dfun(x0)
        delta = abs(fun(x0))
    ax.scatter(x0, f(x0), color="black", s=80)
    return(x0)

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
x = np.arange(-1.5, 1.7, 0.001)
ax.plot(x, f(x))
ax.grid()
x_root = newton_raphson(f, df, -1, 0.1)
fig.savefig("out/newton_raphson_root.pdf")
print(f"Root at: {x_root:.4f}")

### Newton-Raphson

In [ ]:
def ddf(x):
    return 18 * x + 6

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
x = np.arange(-1.5, 1.7, 0.001)
ax.plot(x, f(x))
ax.grid()
x_opt = newton_raphson(df, ddf, 1, 0.1)
fig.savefig("out/newton_raphson_optimum.pdf")
print(f"Minimum at: {x_opt:.4f}")

### Import minimize

In [ ]:
from scipy.optimize import minimize

### 1D optimization using minimize

In [ ]:
def f(x):
    return (x - 4)**2 + 3

x0 = [1]  # the initial guess
result = minimize(f, x0)
result

### 1D optimization using minimize

In [ ]:
min_y = result.fun  # get minimum of the function f
min_x = result.x  # get the x value of the minimum

fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(1, 1, 1)
x = np.arange(1, 7, 0.001)
ax.plot(x, f(x))
ax.scatter(min_x, min_y, color="red", s=120)
fig.savefig("out/minimize_1D.pdf")

### 2D optimization using minimize

In [ ]:
def f(x):
    return (x[0] - 1)**2 + (x[1] - 2.5)**2

x0 = [0, 0]  # the initial guess
result = minimize(f, x0)
result

### Comparison

In [ ]:
def rosen(x):
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

x0 = [1.3, 0.4]  # random initial guess

res_1 = minimize(rosen, x0, method="Nelder-Mead")
res_2 = minimize(rosen, x0, method="Powell")
res_3 = minimize(rosen, x0, method="CG")
res_4 = minimize(rosen, x0, method="BFGS")

### Comparison results

In [ ]:
# The perfect solution would be (1, 1)

res_1.x
res_2.x
res_3.x
res_4.x

### Tin can optimization

In [ ]:
def s(x):
    r = x[0]
    h = x[1]
    return 2 * np.pi * r * (r + h)


def v(x):
    r = x[0]
    h = x[1]
    return np.pi * r**2 * h - 500  # as it is compared to zero

### Constraints

In [ ]:
con = {"type": "eq", "fun": v}

### Tin can optimization

In [ ]:
x0 = [1, 1]
result = minimize(s, x0, method="SLSQP", constraints=con)
result
x = result.x

### Tin can optimization result

In [ ]:
r, h = x
r
h
np.pi * r**2 * h
s(x)